In [2]:
import pandas as pd
import requests
import datetime
import os
from os.path import join, dirname
from dotenv import load_dotenv

In [3]:
twitch_client_id="z501ocurhlyyv5m6zofmtn286z14dl"
twitch_client_secret = "98e30fdh15sxl4swxxrblzjr6vjr33"

def validate_twitch():

    twitch_token_url = "https://id.twitch.tv/oauth2/token"
    twitch_token_data= {
        "grant_type":"client_credentials",
        "client_id": f"{twitch_client_id}",
        "client_secret":f"{twitch_client_secret}",
        "scope":"analytics:read:games"
        }

    twitch_r = requests.post(twitch_token_url,data=twitch_token_data)
    return twitch_r, twitch_client_id

twitch_validation, twitchid = validate_twitch()

def twitch_access_token(twitch_r):
    ## Twitch Status Code
    twitch_status_code = twitch_r.status_code in range(200,299)

    if twitch_status_code:
        twitch_response_data = twitch_r.json()
        ## How much time before it expires?
        twitch_expires_in = twitch_r.json()["expires_in"]
        twitch_now = datetime.datetime.now()
        twitch_expires = twitch_now + datetime.timedelta(seconds=twitch_expires_in)
        twitch_did_expire = twitch_expires < twitch_now

        ## TWITCH ACCESS TOKEN
        twitch_access_token = twitch_response_data["access_token"]

    return twitch_access_token

twitch_access = twitch_access_token(twitch_r = twitch_validation)

In [89]:
def top_streamers(lang,twitch_access_token=twitch_access):
    base_url = "https://api.twitch.tv/helix/streams"
    base_params = {"language":f"{lang}", "first":"100"}

    headers = {
        "Authorization":f"Bearer {twitch_access_token}",
        "Client-Id":f"{twitch_client_id}"
    }

    twitch_request = requests.get(base_url,params=base_params, headers=headers).json()["data"]
    
    vids_data = {
    "viewCounts":pd.DataFrame(twitch_request)["viewer_count"].sum(),
    "top_category":pd.DataFrame(twitch_request)[["game_name"]].groupby("game_name").size().reset_index().sort_values(by=0, ascending=False).reset_index().iloc[0:1,:].transpose().to_dict()[0],
    "top_game":pd.DataFrame(twitch_request)[["game_name","viewer_count"]].groupby("game_name").sum().sort_values(by="viewer_count", ascending=False).reset_index().head(1).transpose().to_dict()[0]
    }


    return twitch_request,vids_data

twitch_request,vids_data = top_streamers(lang="en")

vids_data

[{'id': '43742742492',
  'user_id': '75346877',
  'user_login': 'brawlhalla',
  'user_name': 'Brawlhalla',
  'game_id': '460316',
  'game_name': 'Brawlhalla',
  'type': 'live',
  'title': 'BCX Doubles - North America',
  'viewer_count': 58521,
  'started_at': '2021-11-07T18:59:51Z',
  'language': 'en',
  'thumbnail_url': 'https://static-cdn.jtvnw.net/previews-ttv/live_user_brawlhalla-{width}x{height}.jpg',
  'tag_ids': ['6ea6bca4-4712-4ab9-a906-e3336a9d8039'],
  'is_mature': False},
 {'id': '44350866029',
  'user_id': '207813352',
  'user_login': 'hasanabi',
  'user_name': 'HasanAbi',
  'game_id': '509658',
  'game_name': 'Just Chatting',
  'type': 'live',
  'title': 'OFFICIALLY BACK NO MORE MESSING AROUND!!! FULL BLOWN ACTUAL GOOD STREAMS',
  'viewer_count': 38043,
  'started_at': '2021-11-07T19:29:07Z',
  'language': 'en',
  'thumbnail_url': 'https://static-cdn.jtvnw.net/previews-ttv/live_user_hasanabi-{width}x{height}.jpg',
  'tag_ids': ['6ea6bca4-4712-4ab9-a906-e3336a9d8039'],
  'i

In [79]:
videos = {
    "viewCounts":pd.DataFrame(a)["viewer_count"].sum(),
    "top_category":pd.DataFrame(a)[["game_name"]].groupby("game_name").size().reset_index().sort_values(by=0, ascending=False).reset_index().iloc[0:1,:].transpose().to_dict()[0]
}


In [78]:
pd.DataFrame(a)["viewer_count"].sum()

152405

In [65]:
views_per_category = pd.DataFrame(a)[["game_name","viewer_count"]].groupby("game_name").sum().sort_values(by="viewer_count", ascending=False).reset_index()
count_per_category = pd.DataFrame(a)[["game_name","game_id"]].groupby("game_name").count().sort_values(by="game_id", ascending=False).reset_index()

df = views_per_category.merge(count_per_category, on="game_name", how="inner")


In [76]:
views_per_category = pd.DataFrame(a)[["game_name","viewer_count"]].groupby("game_name").sum().sort_values(by="viewer_count", ascending=False).reset_index().head(1).transpose().to_dict()[0]
views_per_category

{'game_name': 'Minecraft', 'viewer_count': 29596}

In [77]:
pd.DataFrame(a)[["game_name","viewer_count"]].groupby("game_name").sum().sort_values(by="viewer_count", ascending=False).reset_index().head(1).transpose().to_dict()[0]

{'game_name': 'Minecraft', 'viewer_count': 29596}

In [114]:
news = pd.DataFrame(a)
news.sort_values(by="viewer_count", ascending=False).iloc[0:1,:].transpose().to_dict()[0]["user_id"]

'431460701'

In [113]:
def getallchanneldata(userId, twitch_access_token=twitch_access):
    base_url = "https://api.twitch.tv/helix/users"
    base_params = {"id":userId}
    headers = {
        "Authorization":f"Bearer {twitch_access_token}",
        "Client-Id":f"{twitch_client_id}"
    }

    twitch_request = requests.get(base_url,params=base_params, headers=headers).json()

    # tw2 = pd.DataFrame([i for i in twitch_request["data"]])
    return twitch_request

getallchanneldata(userId=news.sort_values(by="viewer_count", ascending=False).iloc[0:1,:].transpose().to_dict()[0]["user_id"])

{'error': 'Bad Request',
 'status': 400,
 'message': 'Invalid login names, emails or IDs in request'}